```
--- Day 16: Ticket Translation ---
As you're walking to yet another connecting flight, you realize that one of the legs of your re-routed trip coming up is on a high-speed train. However, the train ticket you were given is in a language you don't understand. You should probably figure out what it says before you get to the train station after the next flight.

Unfortunately, you can't actually read the words on the ticket. You can, however, read the numbers, and so you figure out the fields these tickets must have and the valid ranges for values in those fields.

You collect the rules for ticket fields, the numbers on your ticket, and the numbers on other nearby tickets for the same train service (via the airport security cameras) together into a single document you can reference (your puzzle input).

The rules for ticket fields specify a list of fields that exist somewhere on the ticket and the valid ranges of values for each field. For example, a rule like class: 1-3 or 5-7 means that one of the fields in every ticket is named class and can be any value in the ranges 1-3 or 5-7 (inclusive, such that 3 and 5 are both valid in this field, but 4 is not).

Each ticket is represented by a single line of comma-separated values. The values are the numbers on the ticket in the order they appear; every ticket has the same format. For example, consider this ticket:

.--------------------------------------------------------.
| ????: 101    ?????: 102   ??????????: 103     ???: 104 |
|                                                        |
| ??: 301  ??: 302             ???????: 303      ??????? |
| ??: 401  ??: 402           ???? ????: 403    ????????? |
'--------------------------------------------------------'
Here, ? represents text in a language you don't understand. This ticket might be represented as 101,102,103,104,301,302,303,401,402,403; of course, the actual train tickets you're looking at are much more complicated. In any case, you've extracted just the numbers in such a way that the first number is always the same specific field, the second number is always a different specific field, and so on - you just don't know what each position actually means!

Start by determining which tickets are completely invalid; these are tickets that contain values which aren't valid for any field. Ignore your ticket for now.

For example, suppose you have the following notes:

class: 1-3 or 5-7
row: 6-11 or 33-44
seat: 13-40 or 45-50

your ticket:
7,1,14

nearby tickets:
7,3,47
40,4,50
55,2,20
38,6,12
It doesn't matter which position corresponds to which field; you can identify invalid nearby tickets by considering only whether tickets contain values that are not valid for any field. In this example, the values on the first nearby ticket are all valid for at least one field. This is not true of the other three nearby tickets: the values 4, 55, and 12 are are not valid for any field. Adding together all of the invalid values produces your ticket scanning error rate: 4 + 55 + 12 = 71.

Consider the validity of the nearby tickets you scanned. What is your ticket scanning error rate?

```

In [1]:
#!fsharp
#r "nuget: FParsec"
open FParsec

type FieldRule = { field: string ; range1 : int * int ; range2 : int * int }

let input = System.IO.File.ReadAllLines "./input.txt"

let ruleLines = Array.sub input 0 20 |> List.ofArray
let myTicket = Array.get input 22
let nearbyTicketLines = Array.sub input 25 (Array.length input - 25) |> List.ofArray

let parseRange : Parser<int * int, unit> =
    pipe2
        (pint32 .>> pchar '-')
        (pint32)
        (fun a b -> (a, b))

let parseRule : Parser<FieldRule, unit> =
    pipe3
        (charsTillString ": " true 100)
        (parseRange .>> pstring " or ")
        (parseRange)
        (fun field r1 r2 -> { field = field ; range1 = r1 ; range2 = r2})

let parseTicket :Parser<int list, unit> =
    sepBy pint32 (pchar ',')    

let rules = 
    List.map (run parseRule) ruleLines 
    |> List.choose (function
    | Success (a, _, _) -> Some a
    | _ -> None
)

let nearbyTickets = 
    List.map (run parseTicket) nearbyTicketLines |>
    List.choose (function
    | Success (a, _, _) -> Some a
    | _ -> None
)

let validForRule (value: int) (rule: FieldRule) =
    let (r1_min, r1_max) = rule.range1
    let (r2_min, r2_max) = rule.range2

    (r1_min <= value && value <= r1_max) || (r2_min <= value && value <= r2_max)

let validForAnyRule (value: int) (rules: FieldRule list) =
    List.exists (fun r -> validForRule value r ) rules

List.map (List.filter (fun v -> validForAnyRule v rules |> not)) nearbyTickets
|> List.sumBy (List.sum)



    

21956

```
--- Part Two ---
Now that you've identified which tickets contain invalid values, discard those tickets entirely. Use the remaining valid tickets to determine which field is which.

Using the valid ranges for each field, determine what order the fields appear on the tickets. The order is consistent between all tickets: if seat is the third field, it is the third field on every ticket, including your ticket.

For example, suppose you have the following notes:

class: 0-1 or 4-19
row: 0-5 or 8-19
seat: 0-13 or 16-19

your ticket:
11,12,13

nearby tickets:
3,9,18
15,1,5
5,14,9
Based on the nearby tickets in the above example, the first position must be row, the second position must be class, and the third position must be seat; you can conclude that in your ticket, class is 12, row is 11, and seat is 13.

Once you work out which field is which, look for the six fields on your ticket that start with the word departure. What do you get if you multiply those six values together?

```

In [1]:
#!fsharp
let validNearbyTickets = 
    List.filter (List.forall (fun v -> validForAnyRule v rules)) nearbyTickets

let ticketColumns = List.transpose validNearbyTickets

let rec validForRules (validFor: FieldRule list) (values: int list) (remaining: FieldRule list) =
    match remaining with
    | r :: rs -> 
        if List.forall (fun v -> validForRule v r) values then
            validForRules (r :: validFor) values rs
        else
            validForRules validFor values rs
    | [] -> validFor

let validRulesForColumns = List.map (fun vs -> validForRules [] vs rules) ticketColumns

let rec combinations (path: FieldRule list ) (possibilities : FieldRule list list) : FieldRule list list =
    match possibilities with
    | (xs) :: remaining -> List.concat (List.map (fun x -> combinations (x :: path) (List.map (List.filter ((<>) x)) remaining) ) xs)
    | [] -> [List.rev path]

let fields = combinations [] validRulesForColumns

let field = List.head fields

let departureIndexes = List.indexed field |> List.filter (fun (_, x) -> x.field.StartsWith "departure")

let myTicketValues = 
    match run parseTicket myTicket with
    | Success (a, _, _) -> a
    | Failure _ -> failwith "Couldn't parse my ticket"

List.indexed myTicketValues
|> List.fold 
    (fun state (ix, a) ->
        if (List.exists (fst >> ((=) ix)) departureIndexes) then
            state * (int64 a)
        else
            state
    ) 1L


3709435214239